<img src="https://raw.githubusercontent.com/comet-ml/opik/main/apps/opik-documentation/documentation/static/img/opik-logo.svg" width="250"/>

# Tracking a Multi-step LLM Chain

In this exercise, you'll track a multi-step LLM chain with Opik. You can use OpenAI or open source models via LiteLLM.

If you have multiple steps in your LLM pipeline, you can use the `track` decorator to log the traces for each step. If OpenAI is called within one of these steps, the LLM call with be associated with that corresponding step:

# Imports & Configuration

In [ ]:
#%pip install opik openai --quiet

In [5]:
from opik import track
import opik
from opik.integrations.openai import track_openai
from openai import OpenAI
import getpass
import os
from dotenv import load_dotenv

In [ ]:
# # opik configs
# if "OPIK_API_KEY" not in os.environ:
#     os.environ["OPIK_API_KEY"] = getpass.getpass("Enter your Opik API key: ")

# opik.configure()

In [6]:
load_dotenv()
os.environ["OPIK_PROJECT_NAME"] = "Multi-step-Chain-Demo"
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
openai_client = track_openai(openai_client)

# Define First Step

In [7]:
@track
def generate_meal(ingredient):
    prompt = f"Generate one example of a meal that can be made with {ingredient}."
    res = openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return res.choices[0].message.content

# Define Second Step

In [8]:
@track
def generate_recipe(meal):
    prompt = f"Generate a step-by-step recipe for {meal}"
    res = openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return res.choices[0].message.content

# Call Chain

In [9]:
@track
def generate_recipe_from_ingredient(ingredient):
    meal = generate_meal(ingredient)
    story = generate_recipe(meal)
    return story

generate_recipe_from_ingredient("garlic")

OPIK: Started logging traces to the "Multi-step-Chain-Demo" project at https://www.comet.com/opik/hmtm/redirect/projects?name=Multi-step-Chain-Demo.


'Ingredients:\n- 8 oz pasta (such as fettuccine or linguine)\n- 1 lb shrimp, peeled and deveined\n- 4 cloves garlic, minced\n- 4 tbsp butter\n- 1 cup heavy cream\n- 1/2 cup grated Parmesan cheese\n- Salt and pepper, to taste\n- Chopped parsley, for garnish\n- Garlic bread, for serving\n\nInstructions:\n1. Cook the pasta according to package instructions until al dente. Drain and set aside.\n2. In a large skillet, melt the butter over medium heat. Add the minced garlic and sauté for 1-2 minutes until fragrant.\n3. Add the shrimp to the skillet and cook until pink and opaque, about 2-3 minutes per side. Remove the shrimp from the skillet and set aside.\n4. Pour the heavy cream into the skillet and bring to a simmer. Stir in the Parmesan cheese and season with salt and pepper.\n5. Add the cooked pasta and shrimp to the skillet and toss to coat everything in the creamy garlic butter sauce.\n6. Cook for an additional 2-3 minutes until everything is heated through and well combined.\n7. Remo

# Try with your own example!

In [ ]:
generate_recipe_from_ingredient(input("Enter an ingredient: "))